In [1]:
import time
import cv2
import tensorflow as tf
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model

In [2]:
camera_port = 0
camera = cv2.VideoCapture(camera_port)
time.sleep(0.1)
return_value, image = camera.read()
cv2.imshow('image', image)
del(camera)

In [3]:
tf.keras.backend.clear_session()

In [4]:
model_name = './car_bus_train2.h5'
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

In [5]:
tf.keras.backend.set_learning_phase(0) 

In [6]:
model = load_model(model_name)

In [7]:
session = tf.keras.backend.get_session()

In [8]:
INPUT_NODE = [t.op.name for t in model.inputs]
OUTPUT_NODE = [t.op.name for t in model.outputs]
print(INPUT_NODE, OUTPUT_NODE)
frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs])

['input_1'] ['conv_81/BiasAdd', 'conv_93/BiasAdd', 'conv_105/BiasAdd']
Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 366 variables.
INFO:tensorflow:Converted 366 variables to const ops.


In [9]:
pb_file = './frozen_model.pb'
output_dir = './model'
img_height = 224
input_shape = [1,img_height,img_height,3]
input_shape_str = str(input_shape).replace(' ','')
input_shape_str

'[1,224,224,3]'

In [10]:
mo_tf_path = '"C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py"'

In [11]:
%run -i {mo_tf_path} --input_model {pb_file} --output_dir {output_dir} --input_shape {input_shape_str} --data_type FP32

[ ERROR ]  
Detected not satisfied dependencies:
	test-generator: not installed, required: 0.1.1

Please install required versions of components or use install_prerequisites script
C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\install_prerequisites\install_prerequisites_tf.bat
Note that install_prerequisites scripts may install additional components.


Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	D:\Workspace\GitHub\ISS-VSE-2019-09-23-IS1FT-CA2-Part1\./frozen_model.pb
	- Path for generated IR: 	D:\Workspace\GitHub\ISS-VSE-2019-09-23-IS1FT-CA2-Part1\./model
	- IR output name: 	frozen_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,224,224,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration fi

SystemExit: 1